In [1]:
!pip install cudaq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 3.2 MB/s eta 0:00:00
  Created wheel for cudaq: filename=cudaq-0.13.0-py3-none-any.whl size=7002 sha256=2b95d4e231301d3db676328ed569e857af4c8228ab2bb7fe0b1dfef76c16b1d1
  Stored in directory: /root/.cache/pip/wheels/fb/8a/10/ee10abd8723a2fbfd7962112962900ec48cbba4de11305846d
Successfully built cudaq


In [2]:
"""
QAOA for LABS - "Bugün Hızlandır" v2.0 🚀
5 kritik düzeltme:
1. JIT warm-up (timing accuracy)
2. Endianness sanity check
3. Cache size limit (RAM safety)
4. int8 dtype + int return
5. NumPy RNG (reproducibility)
"""

import cudaq
import numpy as np
from math import floor
from scipy.optimize import minimize
import time

# Set GPU target
try:
    cudaq.set_target("nvidia")
    print("Using NVIDIA GPU.")
except:
    print("Falling back to CPU.")

# =============================================================================
# FIX 4: FAST ENERGY with int8 dtype
# =============================================================================
def calculate_labs_energy_fast(sequence):
    """Vectorized with int8 for speed."""
    s = np.asarray(sequence, dtype=np.int32)  # FIX 4: int8
    n = len(s)
    energy = np.int64(0)
    for k in range(1, n):
        c_k = np.dot(s[:n-k], s[k:])
        energy += c_k * c_k
    return int(energy)  # FIX 4: explicit int return

# =============================================================================
# FIX 2: ENDIANNESS SANITY CHECK
# =============================================================================
def verify_bitstring_endianness():
    """
    Run once to verify bitstring ordering matches our assumption.
    Tests with N=3 circuit.
    """
    print("\n🔍 Endianness Sanity Check...")

    @cudaq.kernel
    def test_kernel():
        q = cudaq.qvector(3)
        # Prepare |100⟩ state (qubit 0 = |1⟩, others = |0⟩)
        x(q[0])
        mz(q)

    results = cudaq.sample(test_kernel, shots_count=100)
    most_common = max(results.items(), key=lambda x: x[1])[0]

    print(f"  Prepared |100⟩ (q0=1, q1=0, q2=0)")
    print(f"  Got bitstring: '{most_common}'")

    # Check if q0 is rightmost (LSB) or leftmost (MSB)
    if most_common == "001":
        print("  ✅ Endianness: q0 is RIGHTMOST (LSB) → use [::-1]")
        return True  # Need reversal
    elif most_common == "100":
        print("  ✅ Endianness: q0 is LEFTMOST (MSB) → NO reversal needed")
        return False  # No reversal
    else:
        print(f"  ⚠️ Unexpected result, defaulting to [::-1]")
        return True

# Global flag - set once at startup
REVERSE_BITSTRING = None

def bitstring_to_spins(bitstring):
    """Convert bitstring to spins with correct endianness."""
    global REVERSE_BITSTRING
    if REVERSE_BITSTRING is None:
        REVERSE_BITSTRING = verify_bitstring_endianness()

    if REVERSE_BITSTRING:
        return np.array([1 if b == '0' else -1 for b in bitstring[::-1]], dtype=np.int8)
    else:
        return np.array([1 if b == '0' else -1 for b in bitstring], dtype=np.int8)

# =============================================================================
# FIX 3: CACHE WITH SIZE LIMIT
# =============================================================================
class FastEnergyCache:
    """Cache with max size limit to prevent RAM explosion."""

    __slots__ = ['cache', 'hits', 'misses', 'max_size', 'clears']

    def __init__(self, max_size=200_000):
        self.cache = {}
        self.max_size = max_size
        self.hits = 0
        self.misses = 0
        self.clears = 0

    def get_energy(self, bitstring):
        """Fast path with size limit."""
        cached = self.cache.get(bitstring)
        if cached is not None:
            self.hits += 1
            return cached

        self.misses += 1
        spins = bitstring_to_spins(bitstring)
        energy = calculate_labs_energy_fast(spins)

        # FIX 3: Clear if too large
        if len(self.cache) >= self.max_size:
            self.cache.clear()
            self.clears += 1

        self.cache[bitstring] = energy
        return energy

    def get_spins(self, bitstring):
        """Get spins for a bitstring."""
        return bitstring_to_spins(bitstring)

    def stats(self):
        total = self.hits + self.misses
        if total == 0:
            return "Cache: empty"
        rate = 100 * self.hits / total
        clr = f", {self.clears} clears" if self.clears else ""
        return f"Cache: {self.hits}/{total} ({rate:.0f}%){clr}"

# =============================================================================
# INTERACTION GENERATION
# =============================================================================
def get_labs_interactions(N):
    """Generate G2 and G4 interactions."""
    G2 = []
    G4 = []

    for i in range(N - 2):
        k_limit = floor((N - (i + 1)) / 2)
        for k in range(1, k_limit + 1):
            G2.append([i, i + k])

    for i in range(N - 3):
        t_limit = floor((N - (i + 1) - 1) / 2)
        for t in range(1, t_limit + 1):
            k_start = t + 1
            k_end = N - (i + 1) - t
            for k in range(k_start, k_end + 1):
                G4.append([i, i + t, i + k, i + k + t])

    return G2, G4

# =============================================================================
# QAOA KERNELS
# =============================================================================
@cudaq.kernel
def r_zz(theta: float, q0: cudaq.qubit, q1: cudaq.qubit):
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)

@cudaq.kernel
def r_4body_zzzz(theta: float, q0: cudaq.qubit, q1: cudaq.qubit,
                  q2: cudaq.qubit, q3: cudaq.qubit):
    x.ctrl(q0, q1)
    x.ctrl(q1, q2)
    x.ctrl(q2, q3)
    rz(theta, q3)
    x.ctrl(q2, q3)
    x.ctrl(q1, q2)
    x.ctrl(q0, q1)

@cudaq.kernel
def labs_qaoa_kernel(N: int, G2: list[list[int]], G4: list[list[int]],
                     p_layers: int, gammas: list[float], betas: list[float]):
    q = cudaq.qvector(N)
    h(q)

    for layer in range(p_layers):
        gamma = gammas[layer]
        beta = betas[layer]

        for pair in G2:
            r_zz(2.0 * gamma, q[pair[0]], q[pair[1]])

        for quad in G4:
            r_4body_zzzz(2.0 * gamma, q[quad[0]], q[quad[1]],
                         q[quad[2]], q[quad[3]])

        for i in range(N):
            rx(2.0 * beta, q[i])

    mz(q)

@cudaq.kernel
def labs_qaoa_kernel_minibatch(N: int, G2: list[list[int]], G4_subset: list[list[int]],
                                p_layers: int, gammas: list[float], betas: list[float]):
    """Same kernel but accepts a subset of G4 terms."""
    q = cudaq.qvector(N)
    h(q)

    for layer in range(p_layers):
        gamma = gammas[layer]
        beta = betas[layer]

        for pair in G2:
            r_zz(2.0 * gamma, q[pair[0]], q[pair[1]])

        for quad in G4_subset:
            r_4body_zzzz(2.0 * gamma, q[quad[0]], q[quad[1]],
                         q[quad[2]], q[quad[3]])

        for i in range(N):
            rx(2.0 * beta, q[i])

    mz(q)

# =============================================================================
# FAST QAOA OPTIMIZER v2.0
# =============================================================================
class FastQAOAOptimizer:
    """
    QAOA with all optimizations + 5 critical fixes:
    1. JIT warm-up
    2. Endianness verified
    3. Cache size limit
    4. int8 dtype
    5. NumPy RNG
    """

    def __init__(self, N, p_layers=1,
                 shots_init=200, shots_max=500,
                 g4_sample_ratio=0.5,
                 elite_k=50,
                 seed=42):  # FIX 5: reproducibility

        self.N = N
        self.p_layers = p_layers

        # Adaptive shots
        self.shots_init = shots_init
        self.shots_max = shots_max
        self.shots_current = shots_init

        # G4 minibatch
        self.G2, self.G4_full = get_labs_interactions(N)
        self.g4_sample_ratio = g4_sample_ratio
        self.g4_sample_size = max(1, int(len(self.G4_full) * g4_sample_ratio))

        # FIX 5: NumPy RNG for reproducibility
        self.rng = np.random.default_rng(seed)

        # FIX 3: Cache with size limit
        self.energy_cache = FastEnergyCache(max_size=200_000)

        # Elite tracking (top-K)
        self.elite_k = elite_k
        self.elite_solutions = {}

        # Global best
        self.global_best_energy = float('inf')
        self.global_best_params = None
        self.global_best_bitstring = None

        # Tracking
        self.iteration = 0
        self.cost_history = []
        self.best_energy_history = []

        # Timing (excluding warm-up)
        self.total_circuit_time = 0
        self.total_energy_time = 0

        print(f"\n🚀 FastQAOAOptimizer v2.0 initialized:")
        print(f"  N={N}, p={p_layers}, seed={seed}")
        print(f"  G2={len(self.G2)}, G4={len(self.G4_full)} (sample {self.g4_sample_size}/iter)")
        print(f"  Shots: {shots_init} → {shots_max} (adaptive)")
        print(f"  Elite pool: top-{elite_k}")

    # FIX 1: JIT WARM-UP
    def warmup(self):
        """Trigger JIT compilation before timing starts."""
        print("\n⏳ JIT warm-up (compiling kernels)...")
        t0 = time.time()

        gammas = [0.2] * self.p_layers
        betas = [0.4] * self.p_layers
        G4_subset = self._sample_g4()

        # Compile both kernels
        cudaq.sample(
            labs_qaoa_kernel_minibatch,
            self.N, self.G2, G4_subset,
            self.p_layers, gammas, betas,
            shots_count=64
        )
        cudaq.sample(
            labs_qaoa_kernel,
            self.N, self.G2, self.G4_full,
            self.p_layers, gammas, betas,
            shots_count=64
        )

        print(f"  ✅ Warm-up done in {time.time()-t0:.1f}s")

    # FIX 5: NumPy RNG for G4 sampling
    def _sample_g4(self):
        """Random subset of G4 terms using NumPy RNG."""
        if self.g4_sample_ratio >= 1.0:
            return self.G4_full
        idx = self.rng.choice(len(self.G4_full), size=self.g4_sample_size, replace=False)
        return [self.G4_full[i] for i in idx]

    def _update_shots(self):
        """Increase shots as optimization progresses."""
        progress = min(1.0, self.iteration / 50)
        self.shots_current = int(self.shots_init + progress * (self.shots_max - self.shots_init))

    def _update_elite(self, bitstring, energy):
        """Keep only top-K elite solutions."""
        if len(self.elite_solutions) < self.elite_k:
            self.elite_solutions[bitstring] = (energy, self.iteration)
        else:
            worst_bs = max(self.elite_solutions, key=lambda x: self.elite_solutions[x][0])
            worst_energy = self.elite_solutions[worst_bs][0]

            if energy < worst_energy:
                del self.elite_solutions[worst_bs]
                self.elite_solutions[bitstring] = (energy, self.iteration)

    def cost_function(self, params):
        """Evaluate with all optimizations."""
        gammas = list(params[:self.p_layers])
        betas = list(params[self.p_layers:])

        self._update_shots()
        G4_subset = self._sample_g4()

        # Circuit execution (timing is now accurate after warm-up)
        t0 = time.time()
        results = cudaq.sample(
            labs_qaoa_kernel_minibatch,
            self.N, self.G2, G4_subset,
            self.p_layers, gammas, betas,
            shots_count=self.shots_current
        )
        self.total_circuit_time += time.time() - t0

        # Energy calculation
        t0 = time.time()
        total_energy = 0.0
        total_counts = 0

        for bitstring, count in results.items():
            energy = self.energy_cache.get_energy(bitstring)
            total_energy += energy * count
            total_counts += count

            self._update_elite(bitstring, energy)

            if energy < self.global_best_energy:
                self.global_best_energy = energy
                self.global_best_params = np.array(params).copy()
                self.global_best_bitstring = bitstring

        self.total_energy_time += time.time() - t0

        expected_energy = total_energy / total_counts

        self.iteration += 1
        self.cost_history.append(expected_energy)
        self.best_energy_history.append(self.global_best_energy)

        if self.iteration % 10 == 0:
            print(f"  [{self.iteration}] E_avg={expected_energy:.1f}, "
                  f"E_best={self.global_best_energy:.0f}, "
                  f"shots={self.shots_current}, "
                  f"elite={len(self.elite_solutions)}, "
                  f"{self.energy_cache.stats()}")

        return expected_energy

    def optimize(self, method='COBYLA', maxiter=100, initial_params=None):
        """Run optimization with warm-up."""
        # FIX 1: Warm-up before timing
        self.warmup()

        print(f"\n--- Starting Fast QAOA ({method}, maxiter={maxiter}) ---")

        if initial_params is None:
            initial_params = np.concatenate([
                np.linspace(0.1, 0.5, self.p_layers),
                np.linspace(0.3, 0.7, self.p_layers)
            ])

        t_start = time.time()

        result = minimize(
            self.cost_function,
            initial_params,
            method=method,
            options={'maxiter': maxiter}
        )

        total_time = time.time() - t_start

        print(f"\n--- Complete in {total_time:.1f}s ---")
        print(f"  Circuit: {self.total_circuit_time:.1f}s ({100*self.total_circuit_time/total_time:.0f}%)")
        print(f"  Energy:  {self.total_energy_time:.1f}s ({100*self.total_energy_time/total_time:.0f}%)")
        print(f"  Iters: {self.iteration}")
        print(f"  Best: E={self.global_best_energy:.0f}")
        print(f"  {self.energy_cache.stats()}")

        return result

    def get_elite_seeds(self, top_k=20):
        """Get top solutions as MTS seeds."""
        sorted_elite = sorted(
            self.elite_solutions.items(),
            key=lambda x: x[1][0]
        )[:top_k]

        print(f"\n--- Top {min(top_k, len(sorted_elite))} Elite Seeds ---")
        seeds = []
        for i, (bitstring, (energy, _)) in enumerate(sorted_elite):
            spins = self.energy_cache.get_spins(bitstring)
            seeds.append(list(spins))
            if i < 10:
                print(f"  {i+1}. E={energy:.0f}")

        return seeds

    def final_sampling(self, shots=2000):
        """High-shot sampling with best params."""
        if self.global_best_params is None:
            print("Run optimize() first!")
            return []

        print(f"\n--- Final Sampling ({shots} shots, full G4) ---")

        gammas = list(self.global_best_params[:self.p_layers])
        betas = list(self.global_best_params[self.p_layers:])

        results = cudaq.sample(
            labs_qaoa_kernel,
            self.N, self.G2, self.G4_full,
            self.p_layers, gammas, betas,
            shots_count=shots
        )

        new_elites = 0
        for bitstring, count in results.items():
            energy = self.energy_cache.get_energy(bitstring)

            if bitstring not in self.elite_solutions:
                self._update_elite(bitstring, energy)
                new_elites += 1

            if energy < self.global_best_energy:
                self.global_best_energy = energy
                self.global_best_bitstring = bitstring

        print(f"  Found {new_elites} new unique solutions")
        print(f"  Best energy: {self.global_best_energy:.0f}")

        return self.get_elite_seeds()

# =============================================================================
# MAIN
# =============================================================================
if __name__ == "__main__":
    N = 30

    print("=" * 60)
    print(f"🚀 FAST LABS QAOA v2.0 - N={N}")
    print("=" * 60)

    # FIX 2: Verify endianness once at startup
    _ = bitstring_to_spins("000")  # Triggers check

    # Create optimizer
    optimizer = FastQAOAOptimizer(
        N=N,
        p_layers=1,
        shots_init=200,
        shots_max=500,
        g4_sample_ratio=0.5,
        elite_k=50,
        seed=42  # Reproducible
    )

    # Optimize
    result = optimizer.optimize(method='COBYLA', maxiter=80)

    # Final high-quality sampling
    seeds = optimizer.final_sampling(shots=3000)

    print(f"\n{'='*60}")
    print(f"✅ FINAL: Best E = {optimizer.global_best_energy}")
    print(f"   Seeds ready for MTS: {len(seeds)}")
    print(f"{'='*60}")

Using NVIDIA GPU.
🚀 FAST LABS QAOA v2.0 - N=30

🔍 Endianness Sanity Check...
  Prepared |100⟩ (q0=1, q1=0, q2=0)
  Got bitstring: '100'
  ✅ Endianness: q0 is LEFTMOST (MSB) → NO reversal needed

🚀 FastQAOAOptimizer v2.0 initialized:
  N=30, p=1, seed=42
  G2=210, G4=1925 (sample 962/iter)
  Shots: 200 → 500 (adaptive)
  Elite pool: top-50

⏳ JIT warm-up (compiling kernels)...
  ✅ Warm-up done in 16.1s

--- Starting Fast QAOA (COBYLA, maxiter=80) ---
  [10] E_avg=454.6, E_best=139, shots=254, elite=50, Cache: 0/2270 (0%)
  [20] E_avg=405.5, E_best=139, shots=314, elite=50, Cache: 0/5140 (0%)
  [30] E_avg=419.0, E_best=139, shots=374, elite=50, Cache: 0/8610 (0%)

--- Complete in 200.4s ---
  Circuit: 200.0s (100%)
  Energy:  0.4s (0%)
  Iters: 32
  Best: E=139
  Cache: 0/9376 (0%)

--- Final Sampling (3000 shots, full G4) ---
  Found 3000 new unique solutions
  Best energy: 131

--- Top 20 Elite Seeds ---
  1. E=131
  2. E=139
  3. E=139
  4. E=139
  5. E=147
  6. E=147
  7. E=147
  8. 

In [4]:
try:
    import cupy as cp
    DEVICE = "GPU (CuPy)"
except ImportError:
    import numpy as cp
    DEVICE = "CPU (NumPy)"

def calculate_energy_fft(sequences):
    """
    LABS probleminin enerji fonksiyonu: E = sum_{k=1}^{N-1} (C_k)^2
    Wiener-Khinchin teoremi kullanılarak O(N log N) hızında hesaplanır.
    """
    if sequences.ndim == 1:
        sequences = sequences.reshape(1, -1)

    batch_size, N = sequences.shape
    # FFT hızı için en yakın 2'nin kuvvetine tamamla
    n_fft = 2**((2 * N - 1).bit_length())

    # Fourier Dönüşümü ile otokorelasyon hesabı
    f_seq = cp.fft.rfft(sequences, n=n_fft, axis=1)
    autocorr = cp.fft.irfft(f_seq * cp.conj(f_seq), n=n_fft, axis=1)

    # k=1'den N-1'e kadar olan kısımların kareleri toplamı (C_k^2)
    # cp.round kullanıyoruz çünkü FFT float sonuç döner, LABS tam sayıdır.
    energies = cp.sum(cp.round(autocorr[:, 1:N])**2, axis=1)
    return energies

# --- [STEP 3: PARALEL ADAY LİSTELİ TABU SEARCH (EDUCATION)] ---
def tabu_search_gpu(initial_seq, max_iters=30, tabu_tenure=7):
    """
    Yerel arama adımı. GPU'da tüm komşuları aynı anda değerlendirir.
    """
    N = len(initial_seq)
    current_seq = cp.array(initial_seq)
    current_energy = calculate_energy_fft(current_seq)[0]

    best_seq = current_seq.copy()
    best_energy = current_energy
    tabu_list = cp.zeros(N, dtype=cp.int32)

    for it in range(max_iters):
        # Tüm 1-bit flip komşuları tek bir matriste oluştur (N, N)
        neighbors = cp.tile(current_seq, (N, 1))
        diag_indices = cp.arange(N)
        neighbors[diag_indices, diag_indices] *= -1

        # Enerjileri toplu halde (batch) hesapla
        neighbor_energies = calculate_energy_fft(neighbors)

        # Tabu durumu ve Aspiration (En iyiyi geçiyorsa yasağı del)
        is_not_tabu = (tabu_list <= it)
        is_aspiration = (neighbor_energies < best_energy)
        mask = is_not_tabu | is_aspiration

        neighbor_energies[~mask] = cp.inf

        # En iyi hareketi seç
        best_move_idx = cp.argmin(neighbor_energies)
        current_seq = neighbors[best_move_idx]
        current_energy = neighbor_energies[best_move_idx]

        if current_energy < best_energy:
            best_energy = current_energy
            best_seq = current_seq.copy()

        # Hamleyi yasaklılar listesine ekle
        tabu_list[best_move_idx] = it + tabu_tenure

    return best_seq, float(best_energy)

# --- [STEP 4: ANA MEMETİK TABU SEARCH DÖNGÜSÜ] ---
def run_full_mts(N=64, pop_size=20, generations=50, p_mutate=0.2, initial_population = None):
    """
    Memetik algoritmanın ana döngüsü: Combine -> Mutate -> Educate
    """
    # Popülasyonu başlat (İleride buraya Kuantum Seeds eklenebilir)
    if initial_population is not None:
            # Arkadaşından gelen listeyi GPU matrisine çevir
            quantum_pop = cp.array(initial_population)

            # Eğer kuantumdan gelen dizi sayısı pop_size'dan küçükse geri kalanı rastgele doldur
            if len(quantum_pop) < pop_size:
                extra_count = pop_size - len(quantum_pop)
                extra_pop = cp.random.choice(cp.array([-1, 1]), (extra_count, N))
                population = cp.vstack([quantum_pop, extra_pop])
            else:
                # Eğer kuantumdan çok fazla dizi gelirse en iyilerini/ilklerini al
                population = quantum_pop[:pop_size]
    else:
            # Dışarıdan veri gelmezse tamamen rastgele başlat
        population = cp.random.choice(cp.array([-1, 1]), (pop_size, N))

    global_best_seq = None
    global_best_energy = float('inf')
    history = []

    start_time = time.time()

    for gen in range(generations):
        # 1. COMBINE (Crossover): 3 ebeveynden çoğunluk oyu
        idx = cp.random.choice(pop_size, 3, replace=False)
        child = cp.sign(cp.sum(population[idx], axis=0))
        child[child == 0] = 1

        # 2. MUTATE: Rastgele bit çevirme
        if cp.random.random() < p_mutate:
            m_point = cp.random.randint(0, N)
            child[m_point] *= -1

        # 3. EDUCATION: Tabu Search ile yerel iyileştirme
        improved_child, improved_energy = tabu_search_gpu(child)

        # 4. SELECTION: Popülasyondaki en kötünün yerine koy (Steady-state update)
        pop_energies = calculate_energy_fft(population)
        worst_idx = cp.argmax(pop_energies)

        if improved_energy < pop_energies[worst_idx]:
            population[worst_idx] = improved_child

        # Global Best Güncelleme
        if improved_energy < global_best_energy:
            global_best_energy = improved_energy
            global_best_seq = improved_child.copy()

        history.append(global_best_energy)
        if gen % 10 == 0:
            print(f"Gen {gen:03d} | En İyi Enerji: {global_best_energy:.1f}")

    total_duration = time.time() - start_time
    return global_best_seq, global_best_energy, total_duration, history
best_sol, best_en, duration, hist = run_full_mts(N=N, generations=100)

Gen 000 | En İyi Enerji: 83.0
Gen 010 | En İyi Enerji: 83.0
Gen 020 | En İyi Enerji: 83.0
Gen 030 | En İyi Enerji: 75.0
Gen 040 | En İyi Enerji: 75.0
Gen 050 | En İyi Enerji: 75.0
Gen 060 | En İyi Enerji: 75.0
Gen 070 | En İyi Enerji: 75.0
Gen 080 | En İyi Enerji: 75.0
Gen 090 | En İyi Enerji: 75.0


In [11]:
# 5. RUN HYBRID MEMETIC TABU SEARCH
print(f"\n--- Phase 2: Classical Optimization (Hybrid) ---")
print("Starting MTS with Quantum-Seeded Population...")

# Run MTS using the quantum seeds
best_seq, best_energy, final_pop, history = run_full_mts(
    N=N,
    pop_size=len(seeds),   # Match population size to our seeds
    generations=50,
    initial_population=seeds # <--- INJECT QUANTUM DATA HERE
)
# 6. REPORT RESULTS
print("\n--- Final Results ---")
print(f"Best Energy Found: {best_energy}")
print(f"Best Sequence: {best_seq}")



--- Phase 2: Classical Optimization (Hybrid) ---
Starting MTS with Quantum-Seeded Population...
Gen 000 | En İyi Enerji: 99.0
Gen 010 | En İyi Enerji: 75.0
Gen 020 | En İyi Enerji: 75.0
Gen 030 | En İyi Enerji: 75.0
Gen 040 | En İyi Enerji: 75.0

--- Final Results ---
Best Energy Found: 75.0
Best Sequence: [-1 -1 -1 -1 -1 -1 -1  1  1  1 -1 -1 -1  1  1 -1 -1  1 -1 -1  1  1 -1  1
 -1  1 -1 -1  1 -1]
